# Protein Design Agent with Strands

이 노트북에서 우리는 오픈 소스 Strands Agents 프레임워크를 사용하여 단백질 설계 에이전트를 만들어요.

#### Install Strands agents and required dependencies

In [1]:
%pip install strands-agents strands-agents-tools --quiet

Note: you may need to restart the kernel to use updated packages.


#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.37.1** or higher.

In [2]:
%pip show boto3

Name: boto3
Version: 1.37.1
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: amazon-q-developer-jupyterlab-ext, amazon-sagemaker-sql-editor, amazon-sagemaker-sql-execution, amazon_sagemaker_jupyter_ai_q_developer, autogluon.common, autogluon.core, autogluon.multimodal, aws-glue-sessions, langchain-aws, PyAthena, redshift_connector, sagemaker, sagemaker-core, sagemaker-jupyterlab-extension-common, sagemaker-kernel-wrapper, sagemaker-mlflow, sagemaker-studio-analytics-extension, sagemaker-studio-dataengineering-extensions, sagemaker-studio-dataengineering-sessions, sagemaker_studio, sagemaker_studio_sparkmagic_lib, snowflake-connector-python, strands-agents
Note: you may need to restart the kernel to use updated packages.


#### Import required libraries

In [3]:
import boto3
import json
import time
import uuid
import os
from typing import Dict, Any, Optional
from strands import Agent, tool
from strands.models import BedrockModel
from urllib.parse import urlparse

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

## Prerequisites

Run through the notebook environment setup in [00-setup_environment.ipynb](00-setup_environment.ipynb).

Also ensure you have deployed the protein design CloudFormation stack from `stacks/protein_design_stack.yaml`

#### Setup AWS clients and configuration
Define the clients to AWS services that will be used by tools.

In [4]:
# Initialize AWS clients
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')

# Configuration - Update these values based on your CloudFormation stack outputs
STACK_NAME = 'protein-design-stack'  # Your CloudFormation stack name
DEFAULT_WORKFLOW_ID = None  # Will be retrieved from stack outputs
DEFAULT_ROLE_ARN = None     # Will be retrieved from stack outputs
DEFAULT_S3_BUCKET = None    # Will be retrieved from stack outputs

print(f"Region: {region}")
print(f"Account ID: {account_id}")

Region: us-west-2
Account ID: 664263524008


#### Get CloudFormation stack outputs

In [5]:
# Get CloudFormation stack outputs
cf_client = boto3.client('cloudformation')

try:
    response = cf_client.describe_stacks(StackName=STACK_NAME)
    stack = response['Stacks'][0]

    # Get from outputs
    outputs = stack.get('Outputs', [])
    for output in outputs:
        key = output['OutputKey']
        value = output['OutputValue']
        if key == 'WorkflowId':
            DEFAULT_WORKFLOW_ID = value
        elif key == 'WorkflowExecutionRoleArn':
            DEFAULT_ROLE_ARN = value
            
    parameters = stack.get('Parameters', [])
    for param in parameters:
        if param['ParameterKey'] == 'S3BucketName':
            DEFAULT_S3_BUCKET = param['ParameterValue']
            break
    
    print(f"Workflow ID: {DEFAULT_WORKFLOW_ID}")
    print(f"Role ARN: {DEFAULT_ROLE_ARN}")
    print(f"S3 Bucket: {DEFAULT_S3_BUCKET}")
    
except Exception as e:
    print(f"Error getting stack outputs: {e}")
    print("Please update the configuration variables manually")

Workflow ID: 1024576
Role ARN: arn:aws:iam::664263524008:role/protein-design-stack-WorkflowExecutionRole-QppAFVfXxcAy
S3 Bucket: protein-c776afefa57d4cc09a98ec589dd3376a


# Strands Agent Creation
In this section we create the agent using the Strands framework

#### Define agent configuration and instructions

In [6]:
protein_agent_name = 'Protein-Design-Agent-Strands'
protein_agent_description = "Protein design and optimization agent using AWS HealthOmics workflows with Strands framework"
protein_agent_instruction = """
You are a protein design specialist AI that helps researchers optimize protein sequences using directed evolution algorithms.
You can trigger AWS HealthOmics workflows to perform protein sequence optimization and monitor their progress.

Your capabilities include:
1. Starting protein design optimization workflows with custom parameters
2. Monitoring the status of running workflows
3. Retrieving and analyzing results from completed optimizations

When working with protein sequences:
- Validate that sequences contain only valid amino acid letters
- Provide clear explanations of the optimization process
- Help users understand the results and their implications

Always be helpful and provide detailed information about the protein design process.
"""

#### Define tools for Strands agent
These tools will invoke AWS HealthOmics workflows for protein design operations

In [7]:
# Import the updated tools that match the Bedrock agent action groups
from protein_design_tools import trigger_aho_workflow, monitor_aho_workflow

# Set environment variables for the tools to use
#import os
#os.environ['DEFAULT_WORKFLOW_ID'] = DEFAULT_WORKFLOW_ID
#os.environ['DEFAULT_ROLE_ARN'] = DEFAULT_ROLE_ARN
#os.environ['DEFAULT_S3_BUCKET'] = DEFAULT_S3_BUCKET

print("Tools imported successfully:")
print(f"- trigger_aho_workflow: {trigger_aho_workflow.__doc__.split('Args:')[0].strip()}")
print(f"- monitor_aho_workflow: {monitor_aho_workflow.__doc__.split('Args:')[0].strip()}")

Tools imported successfully:
- trigger_aho_workflow: Trigger the AWS HealthOmics workflow for protein design optimization
- monitor_aho_workflow: Monitor the status of a running AWS HealthOmics workflow and retrieve results when complete


#### Create the Strands Agent

In [9]:
# Create the Bedrock model
model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    client=bedrock_client
)

# Create the agent with tools that match the Bedrock agent action groups
protein_agent = Agent(
    system_prompt=protein_agent_instruction,
    model=model,
    tools=[trigger_aho_workflow, monitor_aho_workflow]
)

print(f"Created Protein Design Agent with Strands framework")

Created Protein Design Agent with Strands framework


# Test the Agent
Let's test the protein design agent with some example queries

In [10]:
#https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/logs/
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

#### Test 1: Start a protein optimization

In [11]:
# Test starting a protein optimization
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS"

query = f"Can you optimize this protein sequence: {test_sequence}"

print("Query:", query)

try:
    # Run the agent
    print("\nResponse:")
    protein_agent(query)
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
INFO | strands.telemetry.metrics | Creating Strands MetricsClient
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7fc953e3b0b0>> | streaming messages
DEBUG | strands.models.bedrock | formatting request
DEBUG | strands.models.bedrock | request=<{'modelId': 'anthropic.claude-3-5-sonnet-20241022-v2:0', 'messages': [{'content': [{'text': 'Can you opti

Query: Can you optimize this protein sequence: EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS

Response:
I'll help you optimize this protein sequence using the AWS HealthOmics workflow. I'll first verify the sequence:

The sequence appears to be valid, containing only standard amino acid letters. It's 121 amino acids long and appears to be an antibody variable domain sequence.

I'll trigger the optimization workflow using this sequence as the seed sequence. Since no specific parameters were provided, I'll use the default settings for the optional parameters.
Tool #1: trigger_aho_workflow


DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.tools.executors._executor | tool_use=<{'toolUseId': 'tooluse_9Eit15z1SCyMTZQ4ACnakA', 'name': 'trigger_aho_workflow', 'input': {'seed_sequence': 'EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS'}}> | streaming
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.

The workflow has been successfully started. I'll now monitor its progress to check the status and retrieve results when it's complete.
Tool #2: monitor_aho_workflow


DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.tools.executors._executor | tool_use=<{'toolUseId': 'tooluse_AgA1IVlHRqOeDz8b8pfEXg', 'name': 'monitor_aho_workflow', 'input': {'runId': '6408075'}}> | streaming
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7fc953e3b0b0>> | streaming messages
DEBUG | strands

The optimization workflow is currently running. This process may take some time as it:
1. Uses protein language models to evaluate sequence variants
2. Performs parallel MCMC sampling to explore the sequence space
3. Optimizes for protein stability and function

Once the workflow completes, it will provide optimized sequence variants with potentially improved properties. You can check the status later by asking me to monitor workflow run 6408075. Would you like me to continue monitoring the workflow, or would you prefer to check back later?

DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | message_count=<6>, window_size=<40> | skipping context reduction


#### Test 2: Monitor workflow status

In [13]:
# Test monitoring a workflow (use a run ID from the previous test)
# Replace 'YOUR_RUN_ID' with an actual run ID from the previous test
test_run_id = "6408075"  # Update this with a real run ID

query = f"Check the status of workflow run {test_run_id}"

print("Query:", query)
print("\nResponse:")
response = protein_agent(query)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7fc953e3b0b0>> | streaming messages
DEBUG | strands.models.bedrock | formatting request
DEBUG | strands.models.bedrock | request=<{'modelId': 'anthropic.claude-3-5-sonnet-20241022-v2:0', 'messages': [{'content': [{'text': 'Can you optimize this protein sequence: EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVR

Query: Check the status of workflow run 6408075

Response:
I'll check the current status of the workflow run 6408075.
Tool #3: monitor_aho_workflow


DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.tools.executors._executor | tool_use=<{'toolUseId': 'tooluse_UNCXm2gfQ6WQbnzNQs4cag', 'name': 'monitor_aho_workflow', 'input': {'runId': '6408075'}}> | streaming
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<trigger_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<monitor_aho_workflow> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7fc953e3b0b0>> | streaming messages
DEBUG | strands

The workflow is still in progress (RUNNING status). This is normal as protein optimization involves computationally intensive steps including:
- Running the protein language model predictions
- Performing MCMC sampling across multiple chains
- Evaluating sequence variants

The workflow will continue to run until it completes all requested optimization steps. You can check the status again at any time by asking me to monitor the same run ID (6408075). Would you like me to keep monitoring the workflow,

DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | message_count=<10>, window_size=<40> | skipping context reduction


 or would you prefer to check back later?

#### Test 3: Advanced optimization with custom parameters

In [ ]:
# Test with custom parameters
query = "Run a protein optimization for sequence ACDEFGHIKLMNPQRSTVWY with 20 parallel chains and 200 steps"

print("Query:", query)
print("\nResponse:")
response = protein_agent(query)
print(response.content)

In [ ]:
# metrics: https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/metrics/

# Access metrics through the AgentResult
print(f"Total tokens: {response.metrics.accumulated_usage['totalTokens']}")
print(f"Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
print(f"Tools used: {list(response.metrics.tool_metrics.keys())}")


# Deploy to Amazon Bedrock AgentCore
Deploy the agent to Bedrock AgentCore for production use

In [ ]:
# Install Bedrock AgentCore if not already installed
%pip install bedrock-agentcore bedrock-agentcore-starter-toolkit --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import deploy_agent

# Deploy the agent to Bedrock AgentCore
try:
    deployment_result = deploy_agent(
        agent=protein_agent,
        agent_name=protein_agent_name,
        description=protein_agent_description
    )
    
    print("Agent deployed successfully to Bedrock AgentCore!")
    print(f"Agent ID: {deployment_result.get('agent_id')}")
    print(f"Agent ARN: {deployment_result.get('agent_arn')}")
    
except Exception as e:
    print(f"Deployment failed: {e}")
    print("Make sure you have the necessary permissions and Bedrock AgentCore is available in your region")

# Conclusion

This notebook demonstrates how to migrate the original Bedrock Agent-based Protein Design Agent to use the Strands framework while maintaining full compatibility with the original functionality.

## Migration Summary:

### Original Bedrock Agent Action Groups:
1. **ProteinDesignTriggerWorkflow** - Lambda function: `protein-design-stack-WorkflowTriggerFunction-2KrE8ky1s8p1`
2. **ProteinDesignMonitorRuns** - Lambda function: `protein-design-stack-WorkflowMonitorFunction-wY1yduueAKV5`

### Migrated Strands Tools:
1. **trigger_aho_workflow** - Direct implementation of the trigger Lambda functionality
2. **monitor_aho_workflow** - Direct implementation of the monitor Lambda functionality

## Key Changes from Original Agent:

1. **Framework**: Uses Strands agents instead of Bedrock Agents
2. **Tool Definition**: Tools are defined using `@tool` decorator instead of Lambda functions
3. **Direct Integration**: Direct AWS SDK calls instead of Lambda function invocations
4. **Simplified Architecture**: No need for separate Lambda functions and API Gateway
5. **Parameter Compatibility**: Maintains exact parameter names and types from original Bedrock agent

## Features Maintained:

- Protein sequence optimization using AWS HealthOmics workflows
- Workflow monitoring and status checking
- Results retrieval from S3
- Parameter validation and error handling
- Same function signatures as original Lambda functions
- Compatible with existing CloudFormation infrastructure

## Benefits of Strands Approach:

- **Simplified Deployment**: No need for complex CloudFormation templates for agent infrastructure
- **Direct Control**: Direct access to AWS services without Lambda intermediaries
- **Easier Testing**: Can test tools directly in the notebook environment
- **Better Integration**: Seamless integration with Bedrock AgentCore for production deployment
- **Cost Effective**: Eliminates Lambda function costs for agent operations
- **Faster Execution**: Removes Lambda cold start delays

The migrated agent maintains all the original functionality while providing a more streamlined development and deployment experience. The tools are now directly integrated into the Strands framework, making them easier to maintain and extend.